## **Parte 01**

In [1]:
%load_ext sql

In [2]:
import pandas as pd
import sqlite3

# Carregar os arquivos CSV
clientes = pd.read_csv('Bases/clientes.csv')
vendas = pd.read_csv('Bases/vendas.csv')

# Conectar ao banco de dados SQLite e criar o arquivo
conn = sqlite3.connect('clientes_vendas.db')
clientes.to_sql('clientes', conn, if_exists='replace', index=False)
vendas.to_sql('vendas', conn, if_exists='replace', index=False)
conn.close()


In [3]:
%sql sqlite:///clientes_vendas.db

Connecting to 'sqlite:///clientes_vendas.db'

In [4]:
%%sql
SELECT * FROM clientes;

Running query in 'sqlite:///clientes_vendas.db'

id_cliente,nome_cliente,cidade
1,João Souza,Rio de Janeiro
2,Ana Costa,São Paulo
3,Carlos Lima,Belo Horizonte


In [5]:
%%sql
SELECT * FROM vendas;

Running query in 'sqlite:///clientes_vendas.db'

id_venda,id_cliente,data_venda,valor_total
1,1,2023-05-10,200
2,2,2023-06-12,450
3,1,2023-07-22,130
4,3,2023-08-15,500
5,2,2023-09-05,670


In [6]:
%%sql

-- Seleciona o nome dos clientes e o valor total das vendas de 2023
SELECT clientes.nome_cliente, SUM(vendas.valor_total) AS total_vendas

-- As tabelas 'clientes' e 'vendas' são combinadas usando JOIN
FROM clientes
JOIN vendas ON clientes.id_cliente = vendas.id_cliente

-- Filtra as vendas realizadas apenas entre 1º de janeiro e 31 de dezembro de 2023
WHERE vendas.data_venda BETWEEN '2023-01-01' AND '2023-12-31'

-- Agrupa os resultados por cliente, somando as vendas de cada cliente
GROUP BY clientes.nome_cliente;


Running query in 'sqlite:///clientes_vendas.db'

nome_cliente,total_vendas
Ana Costa,1120
Carlos Lima,500
João Souza,330


In [7]:
%%sql
-- Seleciona a cidade dos clientes e o valor total das vendas em cada cidade
SELECT clientes.cidade, SUM(vendas.valor_total) AS total_vendas

-- Faz o JOIN entre as tabelas clientes e vendas, utilizando o campo id_cliente como chave
FROM clientes
JOIN vendas ON clientes.id_cliente = vendas.id_cliente

-- Agrupa os resultados por cidade, somando as vendas para cada cidade
GROUP BY clientes.cidade

-- Ordena os resultados pelo valor total das vendas em ordem decrescente (da maior para a menor)
ORDER BY total_vendas DESC;

Running query in 'sqlite:///clientes_vendas.db'

cidade,total_vendas
São Paulo,1120
Belo Horizonte,500
Rio de Janeiro,330


In [24]:
%%sql

-- Insere o cliente somente se ele ainda não existir
-- Dessa forma não corremos o riso de duplicar um cliente
INSERT INTO clientes (id_cliente, nome_cliente, cidade)
SELECT 4, 'Otávio Assumpção', 'Campo Grande'
WHERE NOT EXISTS (
    SELECT 1 FROM clientes WHERE id_cliente = 4
);

-- Visualizando 
SELECT * FROM clientes;

Running query in 'sqlite:///clientes_vendas.db'

id_cliente,nome_cliente,cidade
1,João Souza,Rio de Janeiro
2,Ana Costa,São Paulo
3,Carlos Lima,Belo Horizonte
4,Otávio Assumpção,Campo Grande


In [62]:
%%sql

-- Insere uma nova venda para o cliente 4 apenas se o id_venda não existir
-- Dessa forma, não corremos o risco de registrar a mesma venda mais de uma vez
INSERT INTO vendas (id_venda, id_cliente, valor_total, data_venda)
SELECT 6, 4, 280.00, '2024-01-10'
WHERE NOT EXISTS (
    SELECT 1 FROM vendas WHERE id_venda = 6
);

-- Visualizando
SELECT * FROM vendas;

Running query in 'sqlite:///clientes_vendas.db'

id_venda,id_cliente,data_venda,valor_total
1,1,2023-05-10,200
2,2,2023-06-12,450
3,1,2023-07-22,130
4,3,2023-08-15,500
5,2,2023-09-05,670
6,4,2024-01-10,380


In [60]:
%%sql

-- Atualiza o valor total da venda com id_venda 6 para 380
UPDATE vendas
SET valor_total = 380
WHERE id_venda = 6;

-- Visualizando
SELECT * FROM vendas;

Running query in 'sqlite:///clientes_vendas.db'

1 rows affected.

id_venda,id_cliente,data_venda,valor_total
1,1,2023-05-10,200
2,2,2023-06-12,450
3,1,2023-07-22,130
4,3,2023-08-15,500
5,2,2023-09-05,670
6,4,2024-01-10,380


In [63]:
%%sql

-- Retorna as cidades distintas com clientes que fizeram mais de uma venda
SELECT DISTINCT clientes.cidade
FROM clientes
JOIN vendas ON clientes.id_cliente = vendas.id_cliente
GROUP BY clientes.id_cliente, clientes.cidade
HAVING COUNT(vendas.id_venda) > 1;

Running query in 'sqlite:///clientes_vendas.db'

cidade
Rio de Janeiro
São Paulo


## **Parte 2**

In [295]:
import pandas as pd # Já háviamos importando anteriormente, apenas para reforçar

df_vendas = pd.read_csv('Bases/vendas.csv') # Função read.csv() já retona como DataFrame
df_clientes = pd.read_csv('Bases/clientes.csv') # Vamos utilizar essa base nos próximos itens

# Exibir as 5 primeiras linhas do DataFrame de vendas
df_vendas.head() # .head() retorna os 5 primeiros por default


,id_venda,id_cliente,data_venda,valor_total
0,1,1,2023-05-10,200
1,2,2,2023-06-12,450
2,3,1,2023-07-22,130
3,4,3,2023-08-15,500
4,5,2,2023-09-05,670


In [296]:
# Sem criar um novo df e sem editar o df original
df_vendas[df_vendas['valor_total'] > 300]

,id_venda,id_cliente,data_venda,valor_total
1,2,2,2023-06-12,450
3,4,3,2023-08-15,500
4,5,2,2023-09-05,670


In [297]:
df_clientes

,id_cliente,nome_cliente,cidade
0,1,João Souza,Rio de Janeiro
1,2,Ana Costa,São Paulo
2,3,Carlos Lima,Belo Horizonte


In [298]:
df_master = pd.merge(df_vendas, df_clientes, on='id_cliente') # Vamos utilizar posteriormente para salvar os novos inputs

# Optei por criar novos df ao invés de simplesmente filtrar para ser possível adicionar novas informações (próximo item)
df_total_cliente = df_master.groupby(['id_cliente', 'nome_cliente'])['valor_total'].sum().reset_index()
df_total_cliente

,id_cliente,nome_cliente,valor_total
0,1,João Souza,330
1,2,Ana Costa,1120
2,3,Carlos Lima,500


In [299]:
class Cliente:
    def __init__(self, id_cliente=None, nome_cliente=None, cidade=None):
        self.id_cliente = id_cliente
        self.nome_cliente = nome_cliente
        self.cidade = cidade

    def registrar_venda(self, df_master):
        # Pedindo o ID do cliente via input
        self.id_cliente = int(input("Digite o ID do cliente: "))

        # Verifica se o cliente já existe no DataFrame
        if self.id_cliente in df_master['id_cliente'].values:
            print("Cliente encontrado! Insira os detalhes da nova venda.")
            # Para clientes existentes, pede apenas valor da compra e data
            valor_compra = float(input("Digite o valor da nova compra: "))
            data_venda = input("Digite a data da venda (formato YYYY-MM-DD): ")
            self.nome_cliente = df_master.loc[df_master['id_cliente'] == self.id_cliente, 'nome_cliente'].values[0]
            self.cidade = df_master.loc[df_master['id_cliente'] == self.id_cliente, 'cidade'].values[0]
        else:
            print("Novo cliente! Por favor, insira as informações do cliente.")
            # Para novos clientes, pede nome, cidade, valor da compra e data
            self.nome_cliente = input("Digite o nome do cliente: ")
            self.cidade = input("Digite a cidade do cliente: ")
            valor_compra = float(input("Digite o valor da primeira compra: "))
            data_venda = input("Digite a data da venda (formato YYYY-MM-DD): ")

        # Adiciona a venda ao df_master
        df_master = self.adicionar_venda(df_master, valor_compra, data_venda)

        return df_master

    def adicionar_venda(self, df_master, valor_compra, data_venda):
        # Gera o novo id_venda com base no último id_venda do DataFrame
        if len(df_master) > 0:
            novo_id_venda = df_master['id_venda'].max() + 1
        else:
            novo_id_venda = 1

        # Verifica se o cliente já existe no df_master
        if self.id_cliente in df_master['id_cliente'].values:
            # Cliente existente: Adiciona apenas a nova compra
            nova_venda = pd.DataFrame({
                'id_venda': [novo_id_venda],
                'id_cliente': [self.id_cliente],
                'data_venda': [data_venda],
                'valor_total': [valor_compra],
                'nome_cliente': [self.nome_cliente],  # Nome já existente
                'cidade': [self.cidade]  # Cidade já existente
            })
        else:
            # Novo cliente: Adiciona todos os dados relevantes
            nova_venda = pd.DataFrame({
                'id_venda': [novo_id_venda],
                'id_cliente': [self.id_cliente],
                'data_venda': [data_venda],
                'valor_total': [valor_compra],
                'nome_cliente': [self.nome_cliente],
                'cidade': [self.cidade]
            })

        # Concatena a nova venda ao df_master
        df_master = pd.concat([df_master, nova_venda], ignore_index=True)
        return df_master

    def exibir_saldo(self, df_total_cliente):
        # Busca o saldo atualizado do cliente e o nome do cliente a partir do DataFrame
        if self.id_cliente in df_total_cliente['id_cliente'].values:
            saldo = df_total_cliente.loc[df_total_cliente['id_cliente'] == self.id_cliente, 'valor_total'].values[0]
            nome_cliente = df_total_cliente.loc[df_total_cliente['id_cliente'] == self.id_cliente, 'nome_cliente'].values[0]
            print(f"Cliente {nome_cliente} (ID: {self.id_cliente}) tem um saldo de: R$ {saldo}")
        else:
            print(f"Cliente (ID: {self.id_cliente}) não encontrado.")

In [300]:
# Criando uma instância de Cliente e registrando a venda
cliente = Cliente()
df_master = cliente.registrar_venda(df_master)

# Atualizando df_total_cliente
df_total_cliente = df_master.groupby(['id_cliente', 'nome_cliente'])['valor_total'].sum().reset_index()

# Exibindo o saldo do cliente
cliente.exibir_saldo(df_total_cliente)

Cliente encontrado! Insira os detalhes da nova venda.


In [304]:
# Criando uma instância de Cliente e registrando a venda
cliente = Cliente()
df_master = cliente.registrar_venda(df_master)

# Atualizando df_total_cliente
df_total_cliente = df_master.groupby(['id_cliente', 'nome_cliente'])['valor_total'].sum().reset_index()

# Exibindo o saldo do cliente
cliente.exibir_saldo(df_total_cliente)

Novo cliente! Por favor, insira as informações do cliente.
Cliente Otávio Assumpção (ID: 4) tem um saldo de: R$ 500.0


In [305]:
df_total_cliente

,id_cliente,nome_cliente,valor_total
0,1,João Souza,630.0
1,2,Ana Costa,1120.0
2,3,Carlos Lima,500.0
3,4,Otávio Assumpção,500.0


In [306]:
df_master

,id_venda,id_cliente,data_venda,valor_total,nome_cliente,cidade
0,1,1,2023-05-10,200.0,João Souza,Rio de Janeiro
1,2,2,2023-06-12,450.0,Ana Costa,São Paulo
2,3,1,2023-07-22,130.0,João Souza,Rio de Janeiro
3,4,3,2023-08-15,500.0,Carlos Lima,Belo Horizonte
4,5,2,2023-09-05,670.0,Ana Costa,São Paulo
5,6,1,2020-12-28,300.0,João Souza,Rio de Janeiro
6,7,4,2023-15-04,500.0,Otávio Assumpção,Campo Grande


In [307]:
def calcular_media(lista_valores):
    soma = 0
    # Iterando sobre a lista e somando os valores
    for valor in lista_valores:
        soma += valor
    # Calculando a média
    media = soma / len(lista_valores)
    return media

# Exemplo de uso com df_master['valor_total']
valores_vendas = df_master['valor_total'].tolist()

# Chamando a função para calcular a média
media_vendas = calcular_media(valores_vendas)

# Exibindo o resultado
print(f"A média das vendas é: R$ {media_vendas:.2f}")

A média das vendas é: R$ 392.86


In [313]:
def fatorial(n):
    # Caso base: o fatorial de 0 ou 1 é 1
    if n == 0 or n == 1:
        return 1
    else:
        # Chamada recursiva: n * fatorial(n-1)
        return n * fatorial(n - 1)

# Exemplo de uso
numero = float(input('Numero: '))
resultado = fatorial(numero)
print(f"O fatorial de {numero} é {resultado}")

O fatorial de 4.0 é 24.0
